In [ ]:
import cv2
import numpy as np

def detect_eyes_with_contours(image_path):
    # Loading img ---------------------------
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # GBlure --------------------------------
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # edges by Canny ------------------------
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours -------------------------
    contours, contours2 = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:

        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
        x, y, w, h = cv2.boundingRect(approx)

        # Filter by aspect ratio and size to find potential eyes
        if 0.3 < w/h < 3.5 and 30 < w < 100 and 15 < h < 50:
            eye_region = image[y:y+h, x:x+w]

            # Convert color-----------------
            hsv = cv2.cvtColor(eye_region, cv2.COLOR_BGR2HSV)

            # Define brown color range in HSV
            lower_brown = np.array([10, 50, 20])
            upper_brown = np.array([20, 255, 200])

            # Create a mask for brown color
            mask = cv2.inRange(hsv, lower_brown, upper_brown)
            brown_ratio = cv2.countNonZero(mask) / (w * h)

            # Highlight if the region has a significant brown color ratio
            if brown_ratio > 0.3:
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.circle(image, (0,0) ,5,(0, 255, 0), 3)
    cv2.imshow("Detected Eyes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function
detect_eyes_with_contours("brown-eyes.jpg")


In [ ]:
pixel_values = img.reshape((-1, 3))
pixel_values = np.float32(pixel_values)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

# Number of clusters k
k = 3
compactness, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

centers = np.uint8(centers)
segmented_img = centers[labels]

# Reshape back to the original dimension
segmented_img = segmented_img.reshape(img.shape)

# Show the image
displayImage(segmented_img, "3 Contored Img." ,colored=1)

bluredImg = cv2.medianBlur(segmented_img, 9, (5,5))
# displayImage(bluredImg, "3 Contored Img." ,colored=1)

gray = cv2.cvtColor(bluredImg, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray, method=cv2.HOUGH_GRADIENT, dp=1, minDist=20, 
                        param1=100, param2=45, minRadius=1, maxRadius=200)

if circles is not None:
    circles = np.uint16(np.around(circles))

    for i in circles[0, :]:
        center = (i[0], i[1])
        radius = i[2]
        # draw the outer circle
        cv2.circle(img, center, radius, (0, 255, 0), 30)
        # draw circle center
        cv2.circle(img, center, 3, (255, 0, 0), 2)
displayImage(img, "after detection !! " ,colored=1)



